In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


#  Packages & Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
import sklearn as skl
import warnings
import statsmodels.api as sm

from sklearn import datasets, linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.model_selection import cross_val_score, cross_val_predict, ShuffleSplit, validation_curve, cross_validate
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelBinarizer, RobustScaler, LabelEncoder, scale, MinMaxScaler, PolynomialFeatures
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score, classification_report, roc_curve, roc_auc_score, confusion_matrix
from sklearn.decomposition import PCA
from sklearn import datasets
from sklearn.feature_selection import RFE,SelectFromModel
from sklearn.neighbors import KNeighborsRegressor

from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.datasets import make_classification

from xgboost import XGBClassifier 

from scipy.stats import skew, boxcox_normmax
from scipy.special import boxcox1p

warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.simplefilter('ignore')


## Load prepared Data

In [2]:
#Load Datasets
data = pd.read_csv("train_data_prepared.csv")
target = pd.read_csv("labels_prepared.csv")
target=target[["status_group"]]

# Baseline Model (Random Forest)

In [9]:


X_train, X_test, y_train, y_test = train_test_split(data, target, train_size=0.8)

# rf = RFC(n_estimators = 1000, random_state = 42)

# rf.fit(X_train, y_train)

# rf_score = rf.score(X_test,y_test)

# print('Accuracy:', rf_score)


# Multi-class logReg

In [20]:
clf = LogisticRegression(multi_class="ovr").fit(data, target)
clf_Score = clf.score(X_test,y_test)
print('Accuracy:', clf_Score)

Accuracy: 0.7319023569023569


In [14]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB().fit(X_train.drop(['gps_height','latitude'],axis=1), y_train)
mnb.score(X_test.drop(['gps_height','latitude'], axis=1),y_test)

0.4521043771043771

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import KFold

def grid_search(Xtrain,ytrain, k):
     cv = KFold(k)
    rfc= Pipeline((
    ('one_hot', one_hot()),
    ('rfc', RandomForestClassifier())
    ))

    knn = Pipeline((
    ('choose_geo', filter_d()),
    ('knn', KNeighborsClassifier())
    ))

    svc = Pipeline((
    ('svc', SVC()),
    ))

    lr = Pipeline((
    ('lr',LogisticRegression(multi_class="ovr")),
    ))

    rfc_parameters = {
    'rfc__n_estimators': [10, 20, 30],
    'rfc__criterion': ['gini', 'entropy', 'accuracy'],
    'rfc__max_features': [5, 10, 15],
    'rfc__max_depth': ['auto', 'log2', 'sqrt', None]
    }

    knn_parameters = {
    'knn__n_neighbors': [3, 7, 10],
    'knn__weights': ['uniform', 'distance']
    }

    svc_parameters = {
    'svc__C': [0.01, 0.1, 1.0],
    'svc__kernel': ['rbf', 'poly'],
    'svc__gamma': [0.01, 0.1, 1.0],

    }
    lr_parameters = {
    'mnb__alpha': [0.01, 0.1, 1.0]
    }

    pars = [parameters1, parameters2, parameters3, parameters4]
    pips = [pipeline1, pipeline2, pipeline3, pipeline4]
    
    d_pred = {}
    print "starting Gridsearch"
    for i in range(len(pars)):
        gs = GridSearchCV(pips[i], pars[i],cv =  ,  verbose=2, refit=False, n_jobs=-1)
        gs = gs.fit(X_train, y_train)
        d_pred[pips[i]] = gs.best_estimator_
        print "finished Gridsearch"
        print gs.best_score_

In [ ]:
import lightgbm as lgb
from hyperopt import STATUS_OK

N_FOLDS = 10

# Create the dataset
train_set = lgb.Dataset(train_features, train_labels)

def objective(params, n_folds = N_FOLDS):
    """Objective function for Gradient Boosting Machine Hyperparameter Tuning"""
    
    # Perform n_fold cross validation with hyperparameters
    # Use early stopping and evalute based on ROC AUC
    cv_results = lgb.cv(params, train_set, nfold = n_folds, num_boost_round = 10000, 
                        early_stopping_rounds = 100, metrics = 'auc', seed = 50)
  
    # Extract the best score
    best_score = max(cv_results['auc-mean'])
    
    # Loss must be minimized
    loss = 1 - best_score
    
    # Dictionary with information for evaluation
    return {'loss': loss, 'params': params, 'status': STATUS_OK}
